In [6]:
import requests
import pandas as pd
import defs

In [7]:
session = requests.Session()

In [8]:
instrument = 'EUR_USD'
count = 10
granularity = 'H1'

In [11]:
url = f"{defs.OANDA_URL}/instruments/{instrument}/candles"
url

'https://api-fxtrade.oanda.com/v3/instruments/EUR_USD/candles'

In [13]:
params = dict(
    count = count,
    granularity = granularity,
    price = 'MBA'
)
params

{'count': 10, 'granularity': 'H1', 'price': 'MBA'}

In [16]:
response = session.get(url, params=params, headers=defs.SECURE_HEADER)
response.status_code

200

In [21]:
data = response.json()

In [19]:
data.keys()

dict_keys(['instrument', 'granularity', 'candles'])

In [23]:
len(data['candles'])

10

In [25]:
#creating column names for candles dataframe
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']

In [27]:
for price in prices:
    for oh in ohlc:
        print(f'{price}_{oh}')

mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [28]:
data['candles'][0]

{'complete': True,
 'volume': 17120,
 'time': '2022-12-13T15:00:00.000000000Z',
 'bid': {'o': '1.06449', 'h': '1.06498', 'l': '1.06200', 'c': '1.06357'},
 'mid': {'o': '1.06457', 'h': '1.06505', 'l': '1.06208', 'c': '1.06364'},
 'ask': {'o': '1.06465', 'h': '1.06512', 'l': '1.06216', 'c': '1.06370'}}

In [40]:
candle_data = []
for candle in data['candles']:
    if candle['complete'] == False:
              continue
    new_dict = {}
    new_dict['time'] = candle['time']
    new_dict['volume'] = candle['volume']
    for price in prices:
        for oh in ohlc:
            new_dict[f'{price}_{oh}'] = candle[price][oh]
    candle_data.append(new_dict)
print(candle_data[0])

{'time': '2022-12-13T15:00:00.000000000Z', 'volume': 17120, 'mid_o': '1.06457', 'mid_h': '1.06505', 'mid_l': '1.06208', 'mid_c': '1.06364', 'bid_o': '1.06449', 'bid_h': '1.06498', 'bid_l': '1.06200', 'bid_c': '1.06357', 'ask_o': '1.06465', 'ask_h': '1.06512', 'ask_l': '1.06216', 'ask_c': '1.06370'}


In [41]:
candles_df = pd.DataFrame(our_data)
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-12-13T15:00:00.000000000Z,17120,1.06457,1.06505,1.06208,1.06364,1.06449,1.06498,1.06200,1.06357,1.06465,1.06512,1.06216,1.06370
1,2022-12-13T16:00:00.000000000Z,11102,1.06368,1.06588,1.06248,1.06295,1.06362,1.06581,1.06240,1.06288,1.06375,1.06596,1.06254,1.06302
2,2022-12-13T17:00:00.000000000Z,10272,1.06294,1.06382,1.06108,1.06146,1.06288,1.06376,1.06102,1.06139,1.06301,1.06389,1.06115,1.06152
3,2022-12-13T18:00:00.000000000Z,7696,1.06146,1.06280,1.06104,1.06204,1.06140,1.06274,1.06097,1.06196,1.06153,1.06287,1.06111,1.06211
4,2022-12-13T19:00:00.000000000Z,5858,1.06205,1.06410,1.06181,1.06362,1.06197,1.06404,1.06174,1.06356,1.06213,1.06417,1.06188,1.06369
5,2022-12-13T20:00:00.000000000Z,5653,1.06362,1.06382,1.06255,1.06270,1.06355,1.06375,1.06248,1.06262,1.06370,1.06389,1.06262,1.06277
6,2022-12-13T21:00:00.000000000Z,1616,1.06272,1.06332,1.06272,1.06317,1.06264,1.06322,1.06264,1.06308,1.06279,1.06343,1.06279,1.06326
7,2022-12-13T22:00:00.000000000Z,2768,1.06322,1.06354,1.06300,1.06334,1.06308,1.06324,1.06257,1.06322,1.06336,1.06384,1.06323,1.06346
8,2022-12-13T23:00:00.000000000Z,2540,1.06334,1.06364,1.06210,1.06244,1.06322,1.06354,1.06201,1.06235,1.06345,1.06374,1.06218,1.06252


In [44]:
candles_df.to_pickle('EUR_USD_H1.pkl')